<a href="https://colab.research.google.com/github/bperz/big_data/blob/master/lab4/BDA_l4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/My\ Drive
%cd BDA\ resources

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME
# findspark.init("/gdrive/My\ Drive/BDA\ resources")
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

In [0]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

import re, nltk
nltk.download('stopwords')

# Actual work

## Task1

In [0]:
def tokenize_and_rm(data, stopWords):
  text = re.findall(r"[a-zA-Z]+", data)
  lst = [word.lower() for word in text if word not in stopWords and len(word) > 1]
  return lst

In [0]:
stopWords = set(nltk.corpus.stopwords.words('english'))

In [0]:
data = spark.read.text("summa.txt")
data.show(20)

In [0]:
dataRDD = data.select('value').rdd.flatMap(lambda x: x)
dataRDD = dataRDD.map(lambda x: tokenize_and_rm(x, stopWords))

In [0]:
dataRDD.take(20)
# if you want to print whole text
# select.collect() 

In [0]:
filteredRDD = dataRDD.filter(lambda r: r!=[])
shingles = filteredRDD.flatMap(lambda line: [(word, 1) for word in line])

In [0]:
reduced = shingles.reduceByKey(lambda accum, val: accum+val)

In [0]:
sorted_list = sorted(reduced.collect(), key=lambda x: x[1], reverse=True)

In [0]:
cloud_base = {k:v for (k,v) in sorted_list[:80]}

In [0]:
cloud = WordCloud(background_color="white", max_words=100, width=800, height=600)

In [0]:
cloud.generate_from_frequencies(cloud_base)

In [0]:
cloud.to_file('test.png')

## Task2

In [0]:
import pandas as pd
import numpy as np
import time

In [0]:
data = np.random.randint(low=-32768, high=32768, size=(100, 100), dtype=np.int16)

In [0]:
# df = spark.read.load("data.csv",
#                      format="csv", sep=",", inferSchema="true")

In [0]:
rdd = sc.parallelize(data)
rdd = rdd.flatMap(lambda x: x).map(lambda x: (x,1))

In [0]:
reducedRDD = rdd.reduceByKey(lambda acc, n: acc+n)

In [0]:
reducedRDD = reducedRDD.map(lambda x: (int(x[0]), x[1]))

In [0]:
reducedRDD.take(5)

In [0]:
seqOp = (lambda local_result, list_element: [max(local_result[0], list_element[0]),
                                             min(local_result[1], list_element[0]),
                                             local_result[2]*(local_result[5]/(local_result[5]+list_element[1]))+list_element[0]*(list_element[1]/(local_result[5]+list_element[1])),
                                             local_result[3] + [list_element[0]],
                                             local_result[4]+1,
                                             local_result[5]+list_element[1]])

In [0]:
combOp = (lambda local_result_1, local_result_2: [max(local_result_1[0], local_result_2[0]), 
                                                  min(local_result_1[1], local_result_2[1]),
                                                  local_result_1[2]*(local_result_1[5]/(local_result_1[5]+local_result_2[5])) + local_result_2[2]*(local_result_2[5]/(local_result_1[5]+local_result_2[5])),                                                
                                                  local_result_1[3] + local_result_2[3],
                                                  local_result_1[4]+local_result_2[4],
                                                  local_result_1[5]+local_result_2[5]])

In [0]:
agg = reducedRDD.aggregate([0, 0, 0, [], 0, 0], seqOp, combOp)

In [0]:
agg

## Task3

In [0]:
def tokenize_and_rm(data, stopWords):
  text = re.findall(r"[a-zA-Z]+", data)
  lst = [word.lower() for word in text if word not in stopWords and len(word) > 1]
  return lst

In [0]:
stopWords = set(nltk.corpus.stopwords.words('english'))

In [0]:
with open("summa.txt") as file:
  data = file.read()
  data = tokenize_and_rm(data, stopWords)

data_tuples = [(data[i], data[i+1]) for i in range(len(data) - 1)]

In [0]:
dataRDD = sc.parallelize(data_tuples)
mapped1 = dataRDD.map(lambda x: (x,1))
mapped2 = mapped1.map(lambda x: ((x[0][1], x[0][0]),x[1]))
reduced = mapped1.union(mapped2).reduceByKey(lambda acc, n: acc+n)

In [0]:
reduced_2 = reduced.map(lambda x: (x[0][0], [(x[0][1],x[1])]))

In [0]:
joined = reduced_2.reduceByKey(lambda acc, n: acc + n)

In [0]:
joined = joined.map(lambda x: (x[0], sorted(x[1],key=lambda y: y[1], reverse=True)[:5]))

In [0]:
joined.take(5)

generating random paragraph

In [0]:
import random

In [0]:
word_dict = joined.collectAsMap()

In [0]:
paragraph = "the"
prev_word = 'the'
length = 30

for i in range(length-1):
  word = random.choice(word_dict[prev_word])[0]
  paragraph += ' ' + word
  prev_word = word

paragraph